In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
X_train_f_name = (train_df["filename"].values).astype(str)
y_train = train_df.loc[:,"target_00":].values
X_test_f_name = (test_df["filename"].values).astype(str)

In [4]:
X_train=np.array([])
for i in range(len(X_train_f_name)):
    file_name=(7-len(X_train_f_name[i]))*'0'+X_train_f_name[i]
    f = open('D:/data/train/'+file_name, 'r')
    X_train=np.append(X_train, f.read())

In [5]:
X_test=np.array([])
for i in range(len(X_test_f_name)):
    file_name=(7-len(X_test_f_name[i]))*'0'+X_test_f_name[i]
    f = open('D:/data/test/'+file_name, 'r')
    X_test=np.append(X_test, f.read())

In [7]:
import re
def clean_text(text):
    #transform in lowercase
    text=text.lower()
    #suppress whitespaces in the begining and the end 
    text=re.sub(r"^\s+|\s+$", "",text )
    #supress caracters that are neither alphabet neither whitespace
    text=re.sub('[^A-Za-z ]+', '', text)
    return text

In [8]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [9]:
def lemmatize_text(text):
    tokens = nltk.word_tokenize (text)
    text =' '.join([lemmatizer.lemmatize(word) for word in tokens])
    return text

In [10]:
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fatim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [11]:
def strip_stopwords(text, is_lower_case=False):
    tokens = nltk.word_tokenize (text)
    #tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [12]:
def cleaning(liste):
    
    processed_text=[]
    for  i in range (len(liste)):
        text=liste[i]
        
        #supprimer les chiffres et supprimer les carctères qui ne sont ni des alphabets ni des espaces ,et les espaces au début et à la fin ,transformer en minuscule,
        text=clean_text(text)
            
        # supprimer extra newlines
        text = re.sub(r'[\r|\n|\r\n]+', ' ',text)
        
        # raciner le texte
        text= lemmatize_text(text)
    
        # supprimer extra espaces
        text= re.sub(' +', ' ', text)
        
        # supprimer les mots vides
        text = strip_stopwords(text, is_lower_case=True)
            
        processed_text.append(text)
        
    return processed_text

In [16]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\fatim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [24]:
l=list(X_train)
corpus=cleaning(l)

In [25]:
lT=list(X_test)
corpusT=cleaning(lT)

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=10000)
X = vectorizer.fit_transform(corpus)
XT=vectorizer.fit_transform(corpusT)

In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

In [22]:
LR = LogisticRegression(solver='liblinear',random_state=0)
sgd=SGDClassifier()
svc=SVC()

In [26]:
from sklearn.metrics import hamming_loss

In [28]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y_train, test_size=0.33, random_state=42)

In [30]:
clf=OneVsRestClassifier(svc)
clf.fit(X_tr,np.array(y_tr))
y_predict=clf.predict(X_te)
hamming_accuracy=hamming_loss(y_te,y_predict)
print("Clf: ",svc.__class__.__name__)
print("hamming loss :",hamming_accuracy )

Clf:  SVC
hamming loss : 0.05856469355683604


In [31]:
predictions=clf.predict(XT)

In [32]:
columns=['target_00', 'target_01', 'target_02', 'target_03',
       'target_04', 'target_05', 'target_06', 'target_07', 'target_08',
       'target_09', 'target_10', 'target_11', 'target_12', 'target_13',
       'target_14', 'target_15', 'target_16', 'target_17', 'target_18',
       'target_19', 'target_20', 'target_21', 'target_22']

In [33]:
test_df[columns]=predictions

In [34]:
test_df.to_csv("sub5.csv",index=False)